In [1]:
# Setup: versi, path, dan import dasar
import sys, os
print("Python:", sys.version)

# Path dataset
BASE_DIR = r"D:\SKRIPSI\DETEKSI TBC"
DATA_DIR = os.path.join(BASE_DIR, "DATASET")
print("DATA_DIR:", DATA_DIR)

# Install dependensi di dalam notebook (aman di Jupyter)
try:
	import tensorflow as tf
except ModuleNotFoundError:
	import sys
	!{sys.executable} -m pip install tensorflow
	import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models, optimizers

print("TensorFlow:", tf.__version__)



Python: 3.13.7 (tags/v3.13.7:bcee1c3, Aug 14 2025, 14:15:11) [MSC v.1944 64 bit (AMD64)]
DATA_DIR: D:\SKRIPSI\DETEKSI TBC\DATASET


TensorFlow: 2.20.0


In [2]:
# Parameter data & split
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42
VAL_SPLIT = 0.2

train_datagen = ImageDataGenerator(
	preprocessing_function=preprocess_input,
	rotation_range=10,
	width_shift_range=0.05,
	height_shift_range=0.05,
	zoom_range=0.1,
	horizontal_flip=True,
	fill_mode='nearest',
	validation_split=VAL_SPLIT,
)

val_datagen = ImageDataGenerator(
	preprocessing_function=preprocess_input,
	validation_split=VAL_SPLIT,
)

train_gen = train_datagen.flow_from_directory(
	DATA_DIR,
	target_size=IMG_SIZE,
	batch_size=BATCH_SIZE,
	class_mode='binary',
	subset='training',
	seed=SEED,
)

val_gen = val_datagen.flow_from_directory(
	DATA_DIR,
	target_size=IMG_SIZE,
	batch_size=BATCH_SIZE,
	class_mode='binary',
	subset='validation',
	seed=SEED,
)

print("Classes:", train_gen.class_indices)
steps_per_epoch = train_gen.samples // BATCH_SIZE
validation_steps = val_gen.samples // BATCH_SIZE
print("steps_per_epoch:", steps_per_epoch, "validation_steps:", validation_steps)



Found 3035 images belonging to 2 classes.


Found 758 images belonging to 2 classes.


Classes: {'normal': 0, 'tuberculosis': 1}
steps_per_epoch: 94 validation_steps: 23


In [3]:
# Bangun model MobileNetV2 (transfer learning)
base_model = MobileNetV2(
	input_shape=IMG_SIZE + (3,),
	include_top=False,
	weights='imagenet'
)
base_model.trainable = False  # freeze awal

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs, outputs)

model.compile(
	optimizer=optimizers.Adam(learning_rate=1e-3),
	loss='binary_crossentropy',
	metrics=['accuracy']
)
model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [4]:
# Training dengan callback
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

EPOCHS = 20
ckpt_path = os.path.join(BASE_DIR, 'checkpoints', 'mobilenetv2_tbc_best.keras')
os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)

callbacks = [
	EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True),
	ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6),
	ModelCheckpoint(ckpt_path, monitor='val_accuracy', mode='max', save_best_only=True)
]

history = model.fit(
	train_gen,
	steps_per_epoch=steps_per_epoch,
	epochs=EPOCHS,
	validation_data=val_gen,
	validation_steps=validation_steps,
	callbacks=callbacks
)



C:\Users\ahuma\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


 1/94 ━━━━━━━━━━━━━━━━━━━━ 28:46 19s/step - accuracy: 0.3438 - loss: 1.0191

 2/94 ━━━━━━━━━━━━━━━━━━━━ 2:13 1s/step - accuracy: 0.3906 - loss: 0.9549  

 3/94 ━━━━━━━━━━━━━━━━━━━━ 2:06 1s/step - accuracy: 0.4444 - loss: 0.8962

...

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9512 - loss: 0.1458

94/94 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.9461 - loss: 0.1504 - val_accuracy: 0.8451 - val_loss: 0.3496 - learning_rate: 7.8125e-06


Epoch 20/20


 1/94 ━━━━━━━━━━━━━━━━━━━━ 42s 459ms/step - accuracy: 1.0000 - loss: 0.0890

94/94 ━━━━━━━━━━━━━━━━━━━━ 20s 212ms/step - accuracy: 1.0000 - loss: 0.0890 - val_accuracy: 0.8451 - val_loss: 0.3516 - learning_rate: 3.9063e-06


In [5]:
# Evaluasi dan metrik
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# Prediksi pada seluruh validation set
val_gen.reset()
y_true = val_gen.classes
# jumlah langkah untuk cover semua sample val
val_steps_full = int(np.ceil(val_gen.samples / BATCH_SIZE))
preds = model.predict(val_gen, steps=val_steps_full)
y_pred_prob = preds.ravel()
y_pred = (y_pred_prob >= 0.5).astype(int)

print("Akurasi:", (y_pred == y_true).mean())
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=list(train_gen.class_indices.keys())))
try:
	auc = roc_auc_score(y_true, y_pred_prob)
	print("ROC-AUC:", auc)
except Exception as e:
	print("ROC-AUC gagal dihitung:", e)



 1/24 ━━━━━━━━━━━━━━━━━━━━ 1:13 3s/step

 2/24 ━━━━━━━━━━━━━━━━━━━━ 9s 437ms/step

 3/24 ━━━━━━━━━━━━━━━━━━━━ 12s 584ms/step

 4/24 ━━━━━━━━━━━━━━━━━━━━ 11s 587ms/step

 5/24 ━━━━━━━━━━━━━━━━━━━━ 10s 554ms/step

...

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 767ms/step

24/24 ━━━━━━━━━━━━━━━━━━━━ 21s 768ms/step


Akurasi: 0.7915567282321899
Confusion Matrix:
 [[596  22]
 [136   4]]
Classification Report:
               precision    recall  f1-score   support

      normal       0.81      0.96      0.88       618
tuberculosis       0.15      0.03      0.05       140

    accuracy                           0.79       758
   macro avg       0.48      0.50      0.47       758
weighted avg       0.69      0.79      0.73       758



ROC-AUC:

 0.5133726306056403


In [6]:
# Simpan model dan label map + contoh inferensi
LABEL_MAP = {v: k for k, v in train_gen.class_indices.items()}
print(LABEL_MAP)

export_dir = os.path.join(BASE_DIR, 'exports')
os.makedirs(export_dir, exist_ok=True)
model_path = os.path.join(export_dir, 'mobilenetv2_tbc.keras')
labels_path = os.path.join(export_dir, 'labels.json')

model.save(model_path)
import json
with open(labels_path, 'w') as f:
	json.dump(LABEL_MAP, f)

# Contoh inferensi pada satu gambar
import numpy as np
from tensorflow.keras.preprocessing import image

# ganti ke path gambar uji yang Anda inginkan
sample_path = os.path.join(DATA_DIR, 'tuberculosis', os.listdir(os.path.join(DATA_DIR, 'tuberculosis'))[0])
img = image.load_img(sample_path, target_size=IMG_SIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
prob = model.predict(x)[0,0]
cls = 1 if prob >= 0.5 else 0
print("Prediksi:", LABEL_MAP[cls], "Prob:", float(prob))



{0: 'normal', 1: 'tuberculosis'}


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Prediksi: tuberculosis Prob: 0.7281450033187866
